In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\nabul\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Gather latest news information from redplanetscience.com
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all news story elements
results = soup.find_all('div', class_='list_text')

# Iterate through found story elements
for result in results[0]:
    # collect latest news title
    ltst_news_title = soup.find('div', class_='content_title').get_text()
    
    # collect latest news paragraph
    ltst_news_text = soup.find('div', class_='article_teaser_body').get_text()

# Check results
print(ltst_news_title)
print("-------")
print(ltst_news_text)

Mars Is Getting a New Robotic Meteorologist
-------
Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.


In [5]:
# Gather image from current featured mars image from spaceimages-mars.com
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [6]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all news story elements
results = soup.find_all('div', class_='header')

for result in results[0]:
    img = soup.find('img', class_='headerimage')
    src = img['src']

# Store and check the results
featured_image_url = url + src
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [7]:
# Read all tables from 'https://galaxyfacts-mars.com/' into pandas
mars_table = pd.read_html('https://galaxyfacts-mars.com/')

# Determine number of tables grabbed
len(mars_table)

2

In [8]:
# Convert target table into a dataframe to check scrape results
mars_info_df = mars_table[0]
mars_info_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
# Scrape high-res mars images from https://marshemispheres.com/
url = 'https://marshemispheres.com/'
browser.visit(url)

In [10]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all links to images
image_links = soup.find_all('div', class_='item')

link_list = []

for image_link in image_links:
    link = image_link.find('a', class_='itemLink product-item')
    link_list.append(url + link['href'])

link_list

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [11]:
# list to store image information
hemisphere_image_urls = []

for link in link_list:
    
    # go to the url with the hihg-quality image
    url = link
    browser.visit(url)
    html = browser.html
    
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all news story elements
    images_info = soup.find_all('div', class_='cover')
    
    for image_info in images_info:
        title = image_info.find('h2', class_='title').get_text()
        description = image_info.find('div', class_='description')
        a = description.find('a')
        image_url = url + a['href']
        print(f"{title}: {image_url}")

Cerberus Hemisphere Enhanced: https://marshemispheres.com/cerberus.htmlimages/cerberus_enhanced.tif
Schiaparelli Hemisphere Enhanced: https://marshemispheres.com/schiaparelli.htmlimages/schiaparelli_enhanced.tif
Syrtis Major Hemisphere Enhanced: https://marshemispheres.com/syrtis.htmlimages/syrtis_major_enhanced.tif
Valles Marineris Hemisphere Enhanced: https://marshemispheres.com/valles.htmlimages/valles_marineris_enhanced.tif


In [12]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]